# Time Period: 20141231-20150331

In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline

### X1

In [2]:
AShareEODDerivativeIndicator = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/EODDerivativeIndicator.csv", header=0)

In [3]:
ME_20141231_list = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['TRADE_DT'] == 20141231]['S_VAL_MV'] 

In [4]:
len(ME_20141231_list)

272

In [5]:
# name list
S_INFO_WINDCODE_ME = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['TRADE_DT'] == 20141231]['S_INFO_WINDCODE']

In [6]:
ME_df_20141231 = pd.DataFrame({'WINDCODE': S_INFO_WINDCODE_ME,'ME_20141231': ME_20141231_list})

In [7]:
ME_df_20141231[:10] 

,ME_20141231,WINDCODE
39297,1.927208e+06,002465.SZ
39298,3.427982e+06,601118.SH
39299,1.264127e+07,601633.SH
39300,1.848000e+05,600233.SH
39301,8.711400e+06,600795.SH
39302,2.277940e+07,601818.SH
39303,4.441649e+06,603993.SH
39304,3.081545e+06,600827.SH
39305,1.495594e+06,600060.SH
39306,1.334175e+06,600816.SH


In [8]:
AShareBalanceSheet = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/BalanceSheet.csv", header=0)

In [10]:
BE_20140930_df  = AShareBalanceSheet[AShareBalanceSheet['REPORT_PERIOD']==20140930]
BE_20140930_T1 = BE_20140930_df[BE_20140930_df['STATEMENT_TYPE'] == 408001000] # 合并报表 BE

In [11]:
WINDCODE_BE = BE_20140930_T1['S_INFO_WINDCODE']
BE_20140930_list = BE_20140930_T1['TOT_SHRHLDR_EQY_INCL_MIN_INT']

In [12]:
BE_df_20140930 = pd.DataFrame({'WINDCODE': WINDCODE_BE,'BE_20140930': BE_20140930_list})

In [13]:
BEME_ratio_df = BE_df_20140930.merge(ME_df_20141231, left_on='WINDCODE', right_on='WINDCODE', how='inner')

In [14]:
# obtain BE/ME
BEME_ratio = BEME_ratio_df['BE_20140930']/BEME_ratio_df['ME_20141231']

In [15]:
# Add BE/ME ratio as a new column to dataframe BEME_ratio_df
se = pd.Series(BEME_ratio)
BEME_ratio_df["BE_ME_ratio"] = se.values

In [16]:
BEME_ratio_df = BEME_ratio_df[['WINDCODE', 'BE_20140930', 'ME_20141231', 'BE_ME_ratio']] # rearrange the order of columns

In [17]:
BEME_ratio_df[:10]

,WINDCODE,BE_20140930,ME_20141231,BE_ME_ratio
0,000157.SZ,4.187137e+10,5.440404e+06,7696.372911
1,600887.SH,1.812761e+10,8.773294e+06,2066.226630
2,600276.SH,7.839263e+09,5.636965e+06,1390.688545
3,600009.SH,1.826081e+10,3.780692e+06,4830.018407
4,601328.SH,4.578800e+11,5.049865e+07,9067.172343
5,600150.SH,1.989145e+10,5.079741e+06,3915.838712
6,000728.SZ,1.666901e+10,6.122100e+06,2722.760829
7,600036.SH,3.023980e+11,4.183972e+07,7227.533362
8,600660.SH,8.311906e+09,2.431625e+06,3418.251050
9,601998.SH,2.576850e+11,3.808488e+07,6766.070198


In [18]:
def normalize_list(input_list):
    empty_list = [None]*len(input_list)
    for i in range(len(input_list)):
        empty_list[i] = (input_list[i]-input_list.mean())/input_list.std()
    return empty_list

In [19]:
norm_BEME_ratio = normalize_list(BEME_ratio_df['BE_ME_ratio'])

In [20]:
# Add norm_BEME_ratio as a new column to dataframe BEME_ratio_df
se_norm = pd.Series(norm_BEME_ratio)
BEME_ratio_df["norm_BEME_ratio"] = se_norm.values

In [61]:
BEME_ratio_df[:10]

,WINDCODE,BE_20140930,ME_20141231,BE_ME_ratio,norm_BEME_ratio
0,000157.SZ,4.187137e+10,5.440404e+06,7696.372911,1.492577
1,600887.SH,1.812761e+10,8.773294e+06,2066.226630,-0.716115
2,600276.SH,7.839263e+09,5.636965e+06,1390.688545,-0.981127
3,600009.SH,1.826081e+10,3.780692e+06,4830.018407,0.368114
4,601328.SH,4.578800e+11,5.049865e+07,9067.172343,2.030339
5,600150.SH,1.989145e+10,5.079741e+06,3915.838712,0.009483
6,000728.SZ,1.666901e+10,6.122100e+06,2722.760829,-0.458558
7,600036.SH,3.023980e+11,4.183972e+07,7227.533362,1.308653
8,600660.SH,8.311906e+09,2.431625e+06,3418.251050,-0.185719
9,601998.SH,2.576850e+11,3.808488e+07,6766.070198,1.127622


In [22]:
# Save df as csv for 2014 whole year use
BEME_ratio_df.to_csv("BEME_ratio_df_2015.csv")

### X2

In [23]:
os.chdir('/Users/yiliwang/Documents/ACR Internship/data/')
files = [file for file in os.listdir( os.curdir ) if file.endswith(".csv")]
frames = {}
for each_file in files:
    frames[os.path.splitext(each_file)[0]] = pd.read_csv(each_file)

In [24]:
start_close = [None]*len(BEME_ratio_df['WINDCODE'])
end_close = [None]*len(BEME_ratio_df['WINDCODE'])

In [128]:
for i in range(len(BEME_ratio_df['WINDCODE'])):
    company = BEME_ratio_df['WINDCODE'][i]
    if frames[company][frames[company]['TRADE_DT'] == 20140930]['S_DQ_ADJCLOSE'].empty == False:
        start_close[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140930]['S_DQ_ADJCLOSE'])
    end_close[i] = float(frames[company][frames[company]['TRADE_DT'] == 20141231]['S_DQ_ADJCLOSE'])

In [136]:
# There is a problem with start_close: there is an empty cell

In [133]:
com_qtr_return = [None]*len(BEME_ratio_df['WINDCODE']) # create an empty list to store return for individual company

In [134]:
for i in range(len(BEME_ratio_df['WINDCODE'])):
    com_qtr_return[i] = (end_close[i] - start_close[i])/start_close[i]

In [70]:
csi300_IndexData= pd.read_csv("/Users/yiliwang/Documents/ACR Internship/csi_300_IndexData_daily_.csv", header=0)

In [137]:
index_start = float(csi300_IndexData[csi300_IndexData['date'] == '2014-09-30']['last_price']) 
index_end = float(csi300_IndexData[csi300_IndexData['date'] == '2014-12-31']['last_price'].mean())

In [138]:
index_qtr_return = (index_end - index_start)/index_start # calculate the index quarterly return

In [139]:
print index_qtr_return

0.445406648068


In [140]:
diff_return_com_csi300 = [None]*len(BEME_ratio_df['WINDCODE'])

In [141]:
for i in range(len(BEME_ratio_df['WINDCODE'])):
    diff_return_com_csi300[i] = com_qtr_return[i] - index_qtr_return

In [143]:
diff_return_com_csi300[:10]

[-0.0016479583871043535,
 -0.33999812157300857,
 -0.4343527603654525,
 -0.14272642625654214,
 0.14030763764628335,
 -0.4327516028115491,
 0.8279371182335102,
 0.1514835816836801,
 -0.2914123797534762,
 0.34626001859866434]

In [255]:
Weighted_Return = pd.DataFrame({'Company':BEME_ratio_df['WINDCODE'],'start_price': start_close,'end_price': end_close,'qtr_return':com_qtr_return,'diff_csi300_LAST':diff_return_com_csi300})

In [256]:
# Fix the empty cell problem
Weighted_Return = Weighted_Return.drop([87]).reset_index(drop=True)

In [257]:
Weighted_Return['norm_diff_csi300_LAST'] = normalize_list(Weighted_Return['diff_csi300_LAST'])

In [258]:
Weighted_Return.to_csv("LAST_Weighted_Return_2015_Q1.csv")

### X3

In [212]:
dates_trade = frames['000001.SZ']['TRADE_DT'].unique()
dates_trade.sort() # sort the date

In [213]:
start_dateind = list(dates_trade).index(20141231) # index of start date
end_dateind = list(dates_trade).index(20150331) # index of end date

In [214]:
target_dates = dates_trade[start_dateind:end_dateind+1]

In [215]:
target_dates 

array([20141231, 20150105, 20150106, 20150107, 20150108, 20150109,
       20150112, 20150113, 20150114, 20150115, 20150116, 20150119,
       20150120, 20150121, 20150122, 20150123, 20150126, 20150127,
       20150128, 20150129, 20150130, 20150202, 20150203, 20150204,
       20150205, 20150206, 20150209, 20150210, 20150211, 20150212,
       20150213, 20150216, 20150217, 20150225, 20150226, 20150227,
       20150302, 20150303, 20150304, 20150305, 20150306, 20150309,
       20150310, 20150311, 20150312, 20150313, 20150316, 20150317,
       20150318, 20150319, 20150320, 20150323, 20150324, 20150325,
       20150326, 20150327, 20150330, 20150331])

In [216]:
def find_FAST_mean(company): # For a given company, calculate mean of FLOAT_A_SHR_TODAY *close price (unadjusted) on trading days
    FAST_product = [None]*len(target_dates)
    
    for i in range(len(target_dates)):
        close_p = frames[company][frames[company]['TRADE_DT'] == target_dates[i]]['S_DQ_CLOSE'].values # get close price on a particular date
        
        # dataframe of the company
        df_company = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['S_INFO_WINDCODE'] ==company]
        FAST_day = float(df_company[df_company['TRADE_DT']==target_dates[i]]['FLOAT_A_SHR_TODAY']) #FLOAT_A_SHR_TODAY of a particular day
        
        FAST_product[i] = FAST_day*close_p
        
    FAST_mean = np.mean(FAST_product)
    return FAST_mean

In [217]:
BEME_ratio_df_2015 = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Three_Factor_Model/2015_Q1/BEME_ratio_df_2015.csv", header=0)
norm_BEME_ratio_list = list(BEME_ratio_df_2015['norm_BEME_ratio'])

In [219]:
FAST_mean_list = [find_FAST_mean(company) for company in Weighted_Return['Company']]

In [220]:
FAST_mean_df = pd.DataFrame({'Company':Weighted_Return['Company'],'FAST_mean':FAST_mean_list})

In [221]:
FAST_mean_df['FAST_mean_norm'] = normalize_list(FAST_mean_df['FAST_mean'])

In [223]:
FAST_mean_df.to_csv("FAST_mean_2015_Q1.csv") # save it as csv

In [222]:
FAST_mean_df.shape

(271, 3)

#### Y

In [240]:
# Obatin Close Price on starting and end dates for current qtr
start_close_c = [None]*len(Weighted_Return['Company'])
end_close_c = [None]*len(Weighted_Return['Company'])

In [241]:
for i in range(len(Weighted_Return['Company'])):
    company = Weighted_Return['Company'][i]
    start_close_c[i] =float(frames[company][frames[company]['TRADE_DT'] == 20141231]['S_DQ_ADJCLOSE'])
    end_close_c[i] = float(frames[company][frames[company]['TRADE_DT'] ==20150331]['S_DQ_ADJCLOSE'])

In [242]:
com_qtr_return_CURRENT = [None]*len(Weighted_Return['Company']) 

In [243]:
for i in range(len(Weighted_Return['Company'])):
    com_qtr_return_CURRENT[i] = (end_close_c[i] - start_close_c[i])/start_close_c[i]

In [244]:
index_start_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-12-31']['last_price']) 
index_end_c = float(csi300_IndexData[csi300_IndexData['date'] == '2015-03-31']['last_price'].mean()) 

In [246]:
index_qtr_return_CURRENT = (index_end_c - index_start_c)/index_start_c # calculate the index quarterly return

In [247]:
index_qtr_return_CURRENT

0.146766492994139

In [248]:
diff_return_com_csi300_CURRENT = [None]*len(Weighted_Return['Company'])

In [249]:
for i in range(len(Weighted_Return['Company'])):
    diff_return_com_csi300_CURRENT[i] = com_qtr_return_CURRENT[i] - index_qtr_return_CURRENT

In [251]:
Y_2015_Q1 = pd.DataFrame({'Company':Weighted_Return['Company'],'Y':diff_return_com_csi300_CURRENT})

In [252]:
Y_2015_Q1.to_csv("Y_2015_Q1.csv")

In [266]:
del norm_BEME_ratio_list[87] # to keep same length of lists

In [267]:
TFM_2015_Q1 = pd.DataFrame({'X_1':norm_BEME_ratio_list,'X_2':Weighted_Return['norm_diff_csi300_LAST'], 'X_3': FAST_mean_df['FAST_mean_norm'], 'Y':diff_return_com_csi300_CURRENT})

In [268]:
TFM_2015_Q1

,X_1,X_2,X_3,Y
0,1.492577,0.172273,-0.190166,-0.109961
1,-0.716115,-0.546482,0.071973,-0.069223
2,-0.981127,-0.746919,-0.061710,0.083235
3,0.368114,-0.127419,-0.287801,0.083634
4,2.030339,0.473829,0.786992,-0.206827
5,0.009483,-0.743517,-0.124192,0.074619
6,-0.458558,1.934555,-0.051034,-0.047180
7,1.308653,0.497570,1.384712,-0.208240
8,-0.185719,-0.443271,-0.257481,0.231320
9,1.127622,0.911332,0.906995,-0.242697


In [269]:
TFM_2015_Q1.to_csv("TFM_2015_Q1.csv")

In [270]:
def outlier_index(list_name, std_num):
    mean = list_name.mean() # mean
    std = np.std(list_name, axis=0) # standard deviation
    
    upper_b = mean+std_num*std
    lower_b = mean-std_num*std
    
    ind_upper = [i for i in range(len(list_name)) if list_name[i] > upper_b]
    ind_lower = [i for i in range(len(list_name)) if list_name[i] < lower_b]
    outlier_index = ind_upper+ind_lower
    
    return outlier_index

In [271]:
outlier_all =outlier_index(TFM_2015_Q1['X_1'], 5) + outlier_index(TFM_2015_Q1['X_2'], 5)+ outlier_index(TFM_2015_Q1['X_3'], 5)+ outlier_index(TFM_2015_Q1['Y'], 5)

In [272]:
TFM_2015_Q1_NO = TFM_2015_Q1.drop(TFM_2015_Q1.index[outlier_all])

In [273]:
TFM_2015_Q1_NO[:10]

,X_1,X_2,X_3,Y
0,1.492577,0.172273,-0.190166,-0.109961
1,-0.716115,-0.546482,0.071973,-0.069223
2,-0.981127,-0.746919,-0.061710,0.083235
3,0.368114,-0.127419,-0.287801,0.083634
4,2.030339,0.473829,0.786992,-0.206827
5,0.009483,-0.743517,-0.124192,0.074619
6,-0.458558,1.934555,-0.051034,-0.047180
7,1.308653,0.497570,1.384712,-0.208240
8,-0.185719,-0.443271,-0.257481,0.231320
9,1.127622,0.911332,0.906995,-0.242697


In [274]:
TFM_2015_Q1_NO.to_csv("TFM_2015_Q1_NO.csv")

# ************* Result DataFrame: TFM_2015_Q1_NO.csv *************